# Import

In [60]:
import numpy as np
import pandas as pd
from numpy import reshape
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# Mnist dataset

In [61]:
data_train = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")

In [62]:
from matplotlib import pylab, gridspec
from matplotlib import pyplot as plt

# X_train and y_train

In [63]:
X_train = data_train.drop(["label"], axis = 1)
y_train = pd.get_dummies(data_train["label"])

In [85]:
X_train = X_train.values.reshape(X_train.shape[0], 28, 28, 1)


AttributeError: 'numpy.ndarray' object has no attribute 'values'

# Keras CNN model

In [66]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28,1)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) # 2 на 2 матрица
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))  

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [67]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 10, 10, 64)        18496     
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 8, 8, 64)          36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 4, 4, 64)          0         
__________

# Model.fit

In [75]:
lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=0.0001, patience=1, verbose=1)
model.fit(X_train, y_train, batch_size=128, epochs=19, verbose=1, validation_split=0.3, callbacks=[lr_reduce])

Train on 29399 samples, validate on 12601 samples
Epoch 1/19
29399/29399 [==============================] - 43s 1ms/step - loss: 0.0667 - acc: 0.9813 - val_loss: 0.0327 - val_acc: 0.9906
Epoch 2/19
29399/29399 [==============================] - 45s 2ms/step - loss: 0.0657 - acc: 0.9817 - val_loss: 0.0327 - val_acc: 0.9906

Epoch 00002: ReduceLROnPlateau reducing learning rate to 9.999999092680235e-13.
Epoch 3/19
29399/29399 [==============================] - 46s 2ms/step - loss: 0.0655 - acc: 0.9811 - val_loss: 0.0327 - val_acc: 0.9906

Epoch 00003: ReduceLROnPlateau reducing learning rate to 9.9999988758398e-14.
Epoch 4/19
29399/29399 [==============================] - 46s 2ms/step - loss: 0.0646 - acc: 0.9810 - val_loss: 0.0327 - val_acc: 0.9906

Epoch 00004: ReduceLROnPlateau reducing learning rate to 9.999999146890344e-15.
Epoch 5/19
29399/29399 [==============================] - 45s 2ms/step - loss: 0.0634 - acc: 0.9824 - val_loss: 0.0327 - val_acc: 0.9906

Epoch 00005: ReduceLROn

In [76]:
from random import random

In [77]:
X_test = data_test.values.reshape(data_test.shape[0], 28, 28, 1)

# Wrong pred and True predict percent

In [78]:
pred2 = model.predict(X_train).argmax(1)
data_wrong_pred = data_train[data_train["label"] != pred2]

In [79]:
truepredpercent = 100 - len(data_wrong_pred) / len(data_test) * 100
print('True predict percent: ' + str(truepredpercent) + ' %')

True predict percent: 98.85357142857143 %


# Custom mnist dataset

In [1]:
from PIL import Image
from os import listdir
from sklearn.metrics import accuracy_score
import numpy as np
def readImages():
    answers = list()
    true = list()
    for i in range(10):
        l = listdir("./hr/{}/".format(i))
        true.extend([int(x[0]) for x in l])
        for image in l:
            img = np.array([255 - x for x in np.asarray(Image.open("./hr/{}/{}".format(i, image)).convert("L")).reshape(784,)]).reshape(28, 28)
            answers.append(np.argmax(model.predict(np.array([img]).reshape(1, 28, 28, 1))[0]))
    return accuracy_score(true, answers)
k =readImages()

NameError: name 'model' is not defined

In [89]:
def readImages():
    answers = list()
    true = list()
    l = listdir("./hr/12345/")
    true.extend([int(x[0]) for x in l])
    for image in l:
        img = np.array([255 - x for x in np.asarray(Image.open("./hr/12345/{}".format(image)).convert("L")).reshape(784,)]).reshape(28, 28)
        answers.append(np.argmax(model.predict(np.array([img]).reshape(1, 28, 28, 1))[0]))
    return accuracy_score(true, answers)
readImages()

0.82